In [1]:
%reload_ext autoreload
%autoreload 2

import os
import logging

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from numpyro.diagnostics import hpdi

from hbmep.config import Config
from hbmep.model import functional as F
from hbmep.model.utils import Site as site

from hbmep_paper.utils import setup_logging
from models import NelderMeadOptimization

plt.rcParams['svg.fonttype'] = 'none'

logger = logging.getLogger(__name__)

TOML_PATH = "/home/vishu/repos/hbmep-paper/configs/rats/J_RCML_000.toml"
DATA_PATH = "/home/vishu/repos/hbmep-paper/reports/figures/01_Intro/data.csv"
MAT_PATH = "/home/vishu/repos/hbmep-paper/reports/figures/01_Intro/mat.npy"
RESPONSE = ["APB", "ADM"]
FEATURES = ["participant"]
BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/figures/01_Intro"

os.makedirs(BUILD_DIR, exist_ok=True)
setup_logging(dir=BUILD_DIR, fname="core")


2024-01-30 14:55:03,467 - hbmep_paper.utils.utils - INFO - Logging to /home/vishu/repos/hbmep-paper/reports/figures/01_Intro/core.log


In [2]:
config = Config(toml_path=TOML_PATH)
config.BUILD_DIR = BUILD_DIR
config.RESPONSE = RESPONSE
config.FEATURES = FEATURES
config.MCMC_PARAMS["num_warmup"] = 10000
config.MCMC_PARAMS["num_samples"] = 10000

df = pd.read_csv(DATA_PATH)
mat = np.load(MAT_PATH)

logger.info(f"df.shape: {df.shape}")
logger.info(f"mat.shape: {mat.shape}")


2024-01-30 14:55:04,835 - __main__ - INFO - df.shape: (50, 4)
2024-01-30 14:55:04,835 - __main__ - INFO - mat.shape: (50, 428, 2)


In [3]:
model = NelderMeadOptimization(config=config)
df, encoder_dict = model.load(df=df)

minimize_result = model.run_inference(df=df);

# prediction_df = model.make_prediction_dataset(df=df, num=20000)
# # prediction_df = model.make_prediction_dataset(df=df)
# posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)
# model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
# model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2024-01-30 14:55:06,043 - hbmep.model.baseline - INFO - Initialized nelder_mead
2024-01-30 14:55:06,043 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/figures/01_Intro
2024-01-30 14:55:06,044 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/figures/01_Intro
2024-01-30 14:55:06,045 - hbmep.dataset.core - WARNING - Total non-positive observations: 0
2024-01-30 14:55:06,045 - hbmep.dataset.core - WARNING - Total missing observations: 0
2024-01-30 14:55:06,046 - hbmep.dataset.core - INFO - Processing data ...
2024-01-30 14:55:06,046 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


2024-01-30 14:55:19,395 - models - INFO - Optimal params for response APB:
2024-01-30 14:55:19,396 - models - INFO -        message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.0794039052533974
             x: [ 1.328e+02  8.391e-02  2.419e-03  7.966e-01]
           nit: 235
          nfev: 411
 final_simplex: (array([[ 1.328e+02,  8.391e-02,  2.419e-03,  7.966e-01],
                       [ 1.328e+02,  8.391e-02,  2.419e-03,  7.966e-01],
                       ...,
                       [ 1.328e+02,  8.391e-02,  2.418e-03,  7.966e-01],
                       [ 1.328e+02,  8.391e-02,  2.419e-03,  7.966e-01]]), array([ 7.940e-02,  7.940e-02,  7.940e-02,  7.940e-02,
                        7.940e-02]))
2024-01-30 14:55:32,880 - models - INFO - Optimal params for response ADM:
2024-01-30 14:55:32,881 - models - INFO -        message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.2874956432

In [6]:
import pickle

dest = os.path.join(model.build_dir, "minimize_result.pkl")
with open(dest, "wb") as f:
    pickle.dump((minimize_result,), f)

logger.info(f"Saved to {dest}")


2024-01-30 14:56:45,201 - __main__ - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/figures/01_Intro/minimize_result.pkl


In [4]:
model.build_dir

'/home/vishu/repos/hbmep-paper/reports/figures/01_Intro'